In [1]:
import pandas as pd
from IPython.display import clear_output

In [2]:
data_players=pd.read_csv(r'processed_data/player_profiles_clean.csv').iloc[:,1:]
all_games=pd.read_csv(r'processed_data/all_match_91_2020.csv')
ranking_all=pd.read_csv(r'processed_data/ranking_91_2020_clean.csv').iloc[:,1:]

In [3]:
len_ref=len(all_games)
len_ref

106690

First, need to group 'duplicates' in the dataframe

In [4]:
data_players=data_players.drop_duplicates(['player_name','year'])
ranking_all=ranking_all.groupby(['unique_id','player_name','year','week_title']).mean().reset_index()

# Create the ranking of the player for the given match

In [5]:
all_games=all_games.merge(ranking_all[['unique_id','rank_number']],how='left',left_on='id_winner', right_on='unique_id').rename({'rank_number':'rank_winner'},axis=1).drop('unique_id',axis=1)
all_games=all_games.merge(ranking_all[['unique_id','rank_number']],how='left',left_on='id_loser', right_on='unique_id').rename({'rank_number':'rank_loser'},axis=1).drop('unique_id',axis=1)

# Add the height, birthdate, weight, player_hand, handedness, backhand, turned_pro

In [6]:
data_demo_players=data_players.drop_duplicates('player_name')

all_games=all_games.merge(data_demo_players[['turned_pro','birthdate','player_hand','player_ht','weight_kg','handedness','backhand','player_name']]
                ,how='left',left_on='winner', right_on='player_name').rename({'turned_pro':'turned_pro_winner',
                                                                            'birthdate':'birthdate_winner',
                                                                            'player_hand':'player_hand_winner',
                                                                            'player_ht':'winner_height',
                                                                            'weight_kg':'winner_weight',
                                                                            'handedness':'winner_handedness',
                                                                            'backhand':'winner_backhand'},axis=1).drop('player_name',axis=1)

all_games=all_games.merge(data_demo_players[['turned_pro','birthdate','player_hand','player_ht','weight_kg','handedness','backhand','player_name']]
                ,how='left',left_on='loser', right_on='player_name').rename({'turned_pro':'turned_pro_loser',
                                                                            'birthdate':'birthdate_loser',
                                                                            'player_hand':'player_hand_loser',
                                                                            'player_ht':'loser_height',
                                                                            'weight_kg':'loser_weight',
                                                                            'handedness':'loser_handedness',
                                                                            'backhand':'loser_backhand'},axis=1).drop('player_name',axis=1)

# Adding weighted average features based on time before. 

here I did an exponential average with a time of 2. <span class="burk">That can be changed, as well as the sommothing parameter (2 by default in python.</span>

## Games features

In [7]:
def exponential_avg_stats_players(feature_name,player_name,time_period):
    data=data_players[data_players['player_name']==player_name]
    return (data[feature_name].ewm(span=time_period).mean())

exponential_avg_stats_players('ace','piros_z.',2)

11241    26.0
11242     9.5
Name: ace, dtype: float64

In [8]:
list_features_exp_avg=['ace',
       'df', 'svpt', '1stIn', '1stWon', '2ndWon', 'SvGms', 'bpSaved',
       'bpFaced']

for feature in list_features_exp_avg:
    for player in data_players['player_name'].unique():
        data_players.loc[data_players['player_name']==player,'exp_avg_'+feature]=exponential_avg_stats_players(feature,player,3)

We can now add them to the dataset with the game. 

In [9]:
all_games=pd.merge(all_games,data_players[['exp_avg_'+list_features_exp_avg[k] for k in range (len(list_features_exp_avg))]+['year','player_name']],
                how='left',left_on=['year','winner'], right_on=['year','player_name']).drop('player_name',axis=1)
for col in ['exp_avg_'+list_features_exp_avg[k] for k in range (len(list_features_exp_avg))]:
    all_games=all_games.rename({col:col+'_winner'},axis=1)
    
all_games=pd.merge(all_games,data_players[['exp_avg_'+list_features_exp_avg[k] for k in range (len(list_features_exp_avg))]+['year','player_name']],
                how='left',left_on=['year','loser'], right_on=['year','player_name']).drop('player_name',axis=1)
for col in ['exp_avg_'+list_features_exp_avg[k] for k in range (len(list_features_exp_avg))]:
    all_games=all_games.rename({col:col+'_loser'},axis=1)    

In [10]:
for col in ['exp_avg_'+list_features_exp_avg[k] for k in range (len(list_features_exp_avg))]:
    all_games=all_games.rename({col:col+'_winner'},axis=1)

## Ranking features

In [11]:
def exponential_avg_ranking_players(player_name,time_period):
    data=ranking_all[ranking_all['player_name']==player_name]
    return (data['rank_number'].ewm(span=time_period).mean())

exponential_avg_ranking_players('piros_z.',10)

1883276    1754.000000
1883277    1756.750000
1883278    1757.654485
1883279    1756.779950
1883280    1757.704339
              ...     
1883372     736.068545
1883373     718.965173
1883374     703.516960
1883375     689.422967
1883376     646.436973
Name: rank_number, Length: 101, dtype: float64

<span class="burk">Warning: We should work only on the loser players instead of the ranking dataset, since the ranking dataset has many much players than the all_games (13000 vs 3500).</span> 

In [12]:
k=1
for player in all_games['loser'].unique():
    if k%100==0:
        clear_output(wait=True)
        print(k)
    ranking_all.loc[ranking_all['player_name']==player,'exp_avg_ranking']=exponential_avg_ranking_players(player,5)
    k+=1
    

3500


In [13]:
all_games.columns

Index(['year', 'round', 'winner', 'loser', 'games_played', 'court', 'surface',
       'tourney_singles_draw', 'tournoi_unique_id', 'week_title', 'id_winner',
       'id_loser', 'rank_winner', 'rank_loser', 'turned_pro_winner',
       'birthdate_winner', 'player_hand_winner', 'winner_height',
       'winner_weight', 'winner_handedness', 'winner_backhand',
       'turned_pro_loser', 'birthdate_loser', 'player_hand_loser',
       'loser_height', 'loser_weight', 'loser_handedness', 'loser_backhand',
       'exp_avg_ace_winner', 'exp_avg_df_winner', 'exp_avg_svpt_winner',
       'exp_avg_1stIn_winner', 'exp_avg_1stWon_winner',
       'exp_avg_2ndWon_winner', 'exp_avg_SvGms_winner',
       'exp_avg_bpSaved_winner', 'exp_avg_bpFaced_winner', 'exp_avg_ace_loser',
       'exp_avg_df_loser', 'exp_avg_svpt_loser', 'exp_avg_1stIn_loser',
       'exp_avg_1stWon_loser', 'exp_avg_2ndWon_loser', 'exp_avg_SvGms_loser',
       'exp_avg_bpSaved_loser', 'exp_avg_bpFaced_loser'],
      dtype='object')

In [14]:
all_games=all_games.merge(ranking_all[['unique_id','exp_avg_ranking']],how='left',left_on='id_winner',right_on='unique_id').rename({'exp_avg_ranking':'exp_avg_ranking_winner'},axis=1).drop('unique_id',axis=1)
all_games=all_games.merge(ranking_all[['unique_id','exp_avg_ranking']],how='left',left_on='id_loser',right_on='unique_id').rename({'exp_avg_ranking':'exp_avg_ranking_loser'},axis=1).drop('unique_id',axis=1)

# Final Dataset obtaining 

In [17]:
all_games.to_csv(r'processed_data/all_games_with_games_stats.csv',index=False)

# Other features we could add in the future

With Noé, we also thought about other features that we could add in the future. We prefered keeping them in mind but not using them, in order to work on a first dataset, and then improve it. 
Here is a (non exhoaustive) list of them: 
- Nationality/location of the tournament
- Nb tourney won by each pleayer
- slope of the ranking
- relative difference within a year of ranking.

In [301]:
all_games.columns

Index(['year', 'round', 'winner', 'loser', 'games_played', 'court', 'surface',
       'tourney_singles_draw', 'tournoi_unique_id', 'week_title', 'id_winner',
       'id_loser', 'rank_winner', 'rank_loser', 'exp_avg_ace_winner',
       'exp_avg_df_winner', 'exp_avg_svpt_winner', 'exp_avg_1stIn_winner',
       'exp_avg_1stWon_winner', 'exp_avg_2ndWon_winner',
       'exp_avg_SvGms_winner', 'exp_avg_bpSaved_winner',
       'exp_avg_bpFaced_winner', 'exp_avg_ace_loser', 'exp_avg_df_loser',
       'exp_avg_svpt_loser', 'exp_avg_1stIn_loser', 'exp_avg_1stWon_loser',
       'exp_avg_2ndWon_loser', 'exp_avg_SvGms_loser', 'exp_avg_bpSaved_loser',
       'exp_avg_bpFaced_loser', 'exp_avg_ranking_winner',
       'exp_avg_ranking_loser', 'turned_pro_winner', 'birthdate_winner',
       'player_hand_winner', 'winner_height', 'winner_weight',
       'winner_handedness', 'winner_backhand', 'turned_pro_loser',
       'birthdate_loser', 'player_hand_loser', 'loser_height', 'loser_weight',
       'loser_

In [312]:
# Baseline edition

In [311]:
all_games.dropna(subset=['rank_winner','rank_loser'])

,year,round,winner,loser,games_played,court,surface,tourney_singles_draw,tournoi_unique_id,week_title,...,winner_weight,winner_handedness,winner_backhand,turned_pro_loser,birthdate_loser,player_hand_loser,loser_height,loser_weight,loser_handedness,loser_backhand
1,1991,2.0,novacek_k.,vajda_m.,26.0,Outdoor,Hard,32,1991-auckland-hard,1991-01-07,...,82.0,NaN,NaN,NaN,1965.03.24,R,173.0,68.0,NaN,NaN
4,1991,3.0,novacek_k.,jonsson_l.,0.0,Outdoor,Hard,32,1991-auckland-hard,1991-01-07,...,82.0,NaN,NaN,1988.0,1970.06.27,R,188.0,70.0,NaN,NaN
5,1991,3.0,mattar_l.,bergstrom_c.,0.0,Outdoor,Hard,32,1991-auckland-hard,1991-01-07,...,76.0,NaN,NaN,1985.0,1967.07.19,R,180.0,68.0,NaN,NaN
6,1991,3.0,vajda_m.,kuhnen_p.,29.0,Outdoor,Hard,32,1991-auckland-hard,1991-01-07,...,68.0,NaN,NaN,1985.0,1966.02.11,R,190.0,82.0,NaN,NaN
7,1991,4.0,sanchez_e.,washington_m.,32.0,Outdoor,Hard,32,1991-auckland-hard,1991-01-07,...,74.0,NaN,NaN,1989.0,1969.06.20,R,180.0,79.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106685,2020,3.0,dimitrov_g.,wawrinka_s.,20.0,Outdoor,Hard,32,2020-acapulco-hard,2020-02-27,...,80.0,Right-Handed,One-Handed Backhand,2002.0,1985.03.28,R,183.0,81.0,Right-Handed,One-Handed Backhand
106686,2020,3.0,nadal_r.,kwon_s.w.,15.0,Outdoor,Hard,32,2020-acapulco-hard,2020-02-27,...,85.0,Left-Handed,Two-Handed Backhand,NaN,NaN,NaN,NaN,NaN,NaN,NaN
106687,2020,2.0,fritz_t.,isner_j.,29.0,Outdoor,Hard,32,2020-acapulco-hard,2020-02-27,...,84.0,Right-Handed,Two-Handed Backhand,2007.0,1985.04.26,R,206.0,108.0,Right-Handed,Two-Handed Backhand
106688,2020,2.0,nadal_r.,dimitrov_g.,17.0,Outdoor,Hard,32,2020-acapulco-hard,2020-02-27,...,85.0,Left-Handed,Two-Handed Backhand,2008.0,1991.05.16,R,188.0,80.0,Right-Handed,One-Handed Backhand


In [314]:
df=all_games.dropna(subset=['rank_winner','rank_loser'])
nb_match=len(df)
nb_good_rank=len(df[df['rank_winner']<df['rank_loser']])
nb_good_rank/nb_match

0.6478204359128175